In [1]:
import mdtraj as md 
import subprocess

In [ ]:
def assign_protonation(md_traj, resname: str, proton_type: str, resid_list: list):
    '''
    ASP:
        0. Not protonated (charge -1) (ASP)
        1. Protonated (charge 0) (ASPP)
    
    HIS:
        0. H on ND1 only (HSD)
        1. H on NE2 only (HSE)
        2. H on ND1 and NE2 (HSP)
        3. Coupled to Heme (HSD)
    '''
    proton_options = []
    for res in md_traj.residues:
        if res.name == resname:
            if res.resSeq in resid_list:
                proton_options.append(f'{proton_type}\n')
            else:
                proton_options.append('0\n')
    return proton_options

In [2]:
t = md.load_pdb('./apo_4XYJ_MD.pdb')
gmx_options = ['1\n', '1\n'] # force filed: charmm, water: CHARMM-modified TIP3P 
for chain in t.topology.chains:
    gmx_options += assign_protonation(chain, 'ASP', '1', [177, 318, 682])
    gmx_options += assign_protonation(chain, 'HIS', '2', [208, 621, 671])

In [ ]:
# write pdb2gmx_option
with open('pdb2gmx_option.txt','w') as f:
    f.writelines(gmx_options)

In [ ]:
len(gmx_options)

In [ ]:
subprocess.run('cat pdb2gmx_option.txt | xargs -I echo| gmx pdb2gmx -f apo_4XYJ_MD.pdb -o apo_4XYJ_MD_conf.pdb -asp -his > pdb2gmx.log', shell=True)